In [17]:
class D:
	def __get__(self, instance, owner):
		print(f"{self}.__get__({instance}, {owner})")
		return self

class A:
	def f(self):
		print(f"{self} (type: {type(self)})")

	g = D()


In [20]:
a = A()
a.g

<__main__.D object at 0x7f1b4c403a60>.__get__(<__main__.A object at 0x7f1b4f592e80>, <class '__main__.A'>)


In [21]:
a.__dict__['g'] = 10
a.g

10

In [13]:
a = A()
a.f

<bound method A.f of <__main__.A object at 0x7f1b4c453df0>>

In [14]:
a.f.__get__(A(), A)

<bound method A.f of <__main__.A object at 0x7f1b4c453df0>>

In [16]:
A.f.__get__(A)()

<class '__main__.A'> (type: <class 'type'>)


In [19]:
A().g

<__main__.D object at 0x7f1b4c403a60>.__get__(<__main__.A object at 0x7f1b4c453760>, <class '__main__.A'>)


In [38]:
import random
import math

In [56]:

# adapted from https://math.stackexchange.com/questions/1227409/indexing-all-combinations-without-making-list
def C(n,k): #computes nCk, the number of combinations n choose k
    result = 1
    for i in range(n):
        result*=(i+1)
    for i in range(k):
        result//=(i+1)
    for i in range(n-k):
        result//=(i+1)
    return result

C = math.comb
def cgen(i,n,k):
    """
    returns the i-th combination of k numbers chosen from 0,1,2,...,n-1
    """
    mx = C(n,k)
    assert 0 <= i <= mx, f"i={i} must be in [0, {mx}]"
    c = []
    r = i
    j = 0
    for s in range(1,k+1):
        cs = j+1
        while r-C(n-cs,k-s)>0:
            r -= C(n-cs,k-s)
            cs += 1
        c.append(cs-1)
        j = cs
    return c


In [57]:
n = 26
k = 10

mx = C(n,k)
mx

5311735

In [58]:
index = random.randint(0, mx)
index

3543086

In [60]:
cgen(index, n, k)

[2, 3, 5, 13, 15, 16, 18, 19, 20, 25]

In [3]:
class Node:
    def __init__(self, gizmo=None, gadget=None):
        self.gizmo = gizmo
        self.gadget = gadget
        self.events = []  # List of events under this attempt (e.g., failures, successes, cached)
        self.children = []  # List of child attempt nodes

def process_log(log):
    root_nodes = []
    stack = []

    for event in log:
        event_type = event[0]

        if event_type == 'attempt':
            _, gizmo, gadget = event
            node = Node(gizmo=gizmo, gadget=gadget)
            if stack:
                # Add as child of current node
                stack[-1].children.append(node)
            else:
                # Top-level attempt
                root_nodes.append(node)
            stack.append(node)

        elif event_type == 'cached':
            _, gizmo, value = event
            if stack:
                # Add to events of current node
                stack[-1].events.append(('cached', gizmo, value))
            else:
                # Top-level cached event
                root_nodes.append(('cached', gizmo, value))

        elif event_type == 'success':
            _, gizmo, gadget, value = event
            if stack:
                node = stack.pop()
                node.events.append(('success', gizmo, value))
            else:
                root_nodes.append(('success', gizmo, value))

        elif event_type == 'failure':
            _, gizmo, gadget, error = event
            if stack:
                node = stack[-1]
                node.events.append(('failure', gizmo, error))
                # Do not pop from the stack; attempt may continue
            else:
                root_nodes.append(('failure', gizmo, error))

        elif event_type == 'missing':
            _, gizmo = event
            if stack:
                node = stack[-1]
                node.events.append(('missing', gizmo))
                # Optionally handle the missing event
            else:
                root_nodes.append(('missing', gizmo))

    return root_nodes

def print_node(node, prefix='', is_last=True):
    # Determine the connector
    connector = '└── ' if is_last else '├── '

    # Print the attempt
    print(prefix + connector + 'Attempt: {} via {}'.format(node.gizmo, node.gadget))

    # Prepare the new prefix for child nodes
    if is_last:
        child_prefix = prefix + '    '
    else:
        child_prefix = prefix + '│   '

    # Combine events and children into one list, preserving order
    items = []

    # Add events first
    for event in node.events:
        items.append(event)

    # Then add child attempts
    for child in node.children:
        items.append(child)

    # Now, print each item
    count = len(items)
    for i, item in enumerate(items):
        item_is_last = (i == count -1)
        if isinstance(item, Node):
            # It's a child node
            print_node(item, child_prefix, item_is_last)
        else:
            # It's an event
            event_type = item[0]
            connector = '└── ' if item_is_last else '├── '
            if event_type == 'failure':
                _, gizmo, error = item
                print(child_prefix + connector + 'Failure: {}, Error: {}'.format(gizmo, error))
            elif event_type == 'success':
                _, gizmo, value = item
                print(child_prefix + connector + 'Success: {} = {}'.format(gizmo, value))
            elif event_type == 'cached':
                _, gizmo, value = item
                print(child_prefix + connector + 'Cached: {} = {}'.format(gizmo, value))
            elif event_type == 'missing':
                _, gizmo = item
                print(child_prefix + connector + 'Missing: {}'.format(gizmo))
            else:
                print(child_prefix + connector + '{}: {}'.format(event_type, gizmo))

# Example log data
log = [
    ('attempt', 'c', '_ToolSkill(c)'),
    ('attempt', 'b', 'AutoToolCraft(b)'),
    ('failure', 'b', 'AutoToolCraft(b)', 'GadgetFailure(None)'),
    ('attempt', 'b', '_ToolSkill(b)'),
    ('attempt', 'a', '_ToolSkill(a)'),
    ('success', 'a', '_ToolSkill(a)', 10),
    ('success', 'b', '_ToolSkill(b)', 20),
    ('cached', 'd', 5),
    ('success', 'c', '_ToolSkill(c)', 15),
    ('attempt', 'a', '_ToolSkill(a)'),
    ('success', 'a', '_ToolSkill(a)', 10),
    ('attempt', 'b', 'AutoToolCraft(b)'),
    ('failure', 'b', 'AutoToolCraft(b)', 'GadgetFailure(None)'),
    ('attempt', 'b', '_ToolSkill(b)'),
    ('cached', 'a', 10),
    ('success', 'b', '_ToolSkill(b)', 20),
    ('attempt', 'c', '_ToolSkill(c)'),
    ('cached', 'b', 20),
    ('missing', 'd'),
    ('cached', 'a', 10),
]

# Process the log and build the tree
root_nodes = process_log(log)

# Print the tree structure
for i, node in enumerate(root_nodes):
    is_last = (i == len(root_nodes) - 1)
    if isinstance(node, Node):
        print_node(node, '', is_last)
    else:
        # It's an event at the root level
        event_type = node[0]
        connector = '└── ' if is_last else '├── '
        if event_type == 'cached':
            _, gizmo, value = node
            print(connector + 'Cached: {} = {}'.format(gizmo, value))
        elif event_type == 'failure':
            _, gizmo, error = node
            print(connector + 'Failure: {}, Error: {}'.format(gizmo, error))
        elif event_type == 'success':
            _, gizmo, value = node
            print(connector + 'Success: {} = {}'.format(gizmo, value))
        elif event_type == 'missing':
            _, gizmo = node
            print(connector + 'Missing: {}'.format(gizmo))
        else:
            print(connector + '{}: {}'.format(event_type, gizmo))


└── Attempt: c via _ToolSkill(c)
    ├── Attempt: b via AutoToolCraft(b)
    │   ├── Failure: b, Error: GadgetFailure(None)
    │   ├── Cached: d = 5
    │   ├── Success: c = 15
    │   └── Attempt: b via _ToolSkill(b)
    │       ├── Success: b = 20
    │       └── Attempt: a via _ToolSkill(a)
    │           └── Success: a = 10
    ├── Attempt: a via _ToolSkill(a)
    │   └── Success: a = 10
    └── Attempt: b via AutoToolCraft(b)
        ├── Failure: b, Error: GadgetFailure(None)
        ├── Attempt: b via _ToolSkill(b)
        │   ├── Cached: a = 10
        │   └── Success: b = 20
        └── Attempt: c via _ToolSkill(c)
            ├── Cached: b = 20
            ├── Missing: d
            └── Cached: a = 10


In [3]:
def visualize_log(log):
    stack = []  # Keeps track of open attempts
    indents = []  # Tracks the indentation and pipe characters for each level

    for event in log:
        event_type = event[0]

        if event_type == 'attempt':
            _, gizmo, gadget = event
            # Push the attempt onto the stack
            node = {'gizmo': gizmo, 'gadget': gadget, 'children': []}
            if stack:
                # Append to the children of the parent node
                stack[-1]['children'].append(node)
            stack.append(node)
            # Update indents for the new level
            if len(stack) == 1:
                indents.append('')
            else:
                # Determine if the parent has more children to decide the pipe character
                if len(stack[-2]['children']) > 1:
                    indents.append('│   ')
                else:
                    indents.append('    ')
            # Print the attempt with '┌── ' to start the pipe
            prefix = ''.join(indents[:-1]) + '┌── '
            print(prefix + 'Attempt: {} via {}'.format(gizmo, gadget))

        elif event_type == 'cached':
            _, gizmo, value = event
            # Print the cached event at the current indentation level
            prefix = ''.join(indents) + ('├── ' if stack else '')
            print(prefix + 'Cached: {} = {}'.format(gizmo, value))

        elif event_type == 'success':
            _, gizmo, gadget, value = event
            # Pop the attempt from the stack
            node = stack.pop()
            # Adjust indents
            if stack:
                indents.pop()
            # Print the success with '└── ' to close the pipe
            prefix = ''.join(indents) + ('└── ' if stack else '└── ')
            print(prefix + 'Success: {} = {}'.format(gizmo, value))

        elif event_type == 'failure':
            _, gizmo, gadget, error = event
            # Pop the attempt from the stack
            node = stack.pop()
            # Adjust indents
            if stack:
                indents.pop()
            # Print the failure with '└── ' to close the pipe
            prefix = ''.join(indents) + ('└── ' if stack else '└── ')
            print(prefix + 'Failure: {}, Error: {}'.format(gizmo, error))

        elif event_type == 'missing':
            _, gizmo = event
            # Print the missing event
            prefix = ''.join(indents) + '├── '
            print(prefix + 'Missing: {}'.format(gizmo))
            if stack:
                # Pop the parent attempt from the stack due to failure
                node = stack.pop()
                if stack:
                    indents.pop()
                # Print the failure of the parent attempt
                prefix = ''.join(indents) + ('└── ' if stack else '└── ')
                print(prefix + 'Failure: {}, Error: Missing dependency: {}'.format(node['gizmo'], gizmo))

log = [
    ('attempt', 'c', '_ToolSkill(c)'),
    ('attempt', 'b', 'AutoToolCraft(b)'),
    ('failure', 'b', 'AutoToolCraft(b)', 'GadgetFailure(None)'),
    ('attempt', 'b', '_ToolSkill(b)'),
    ('attempt', 'a', '_ToolSkill(a)'),
    ('success', 'a', '_ToolSkill(a)', 10),
    ('success', 'b', '_ToolSkill(b)', 20),
    ('cached', 'd', 5),
    ('success', 'c', '_ToolSkill(c)', 15),
    ('attempt', 'a', '_ToolSkill(a)'),
    ('success', 'a', '_ToolSkill(a)', 10),
    ('attempt', 'b', 'AutoToolCraft(b)'),
    ('failure', 'b', 'AutoToolCraft(b)', 'GadgetFailure(None)'),
    ('attempt', 'b', '_ToolSkill(b)'),
    ('cached', 'a', 10),
    ('success', 'b', '_ToolSkill(b)', 20),
    ('attempt', 'c', '_ToolSkill(c)'),
    ('cached', 'b', 20),
    ('missing', 'd'),
    ('cached', 'a', 10),
]

# Visualize the log
visualize_log(log)

visualize_log(log)

┌── Attempt: c via _ToolSkill(c)
┌── Attempt: b via AutoToolCraft(b)
└── Failure: b, Error: GadgetFailure(None)
┌── Attempt: b via _ToolSkill(b)
│   ┌── Attempt: a via _ToolSkill(a)
│   └── Success: a = 10
└── Success: b = 20
├── Cached: d = 5
└── Success: c = 15
┌── Attempt: a via _ToolSkill(a)
└── Success: a = 10
┌── Attempt: b via AutoToolCraft(b)
└── Failure: b, Error: GadgetFailure(None)
┌── Attempt: b via _ToolSkill(b)
├── Cached: a = 10
└── Success: b = 20
┌── Attempt: c via _ToolSkill(c)
├── Cached: b = 20
├── Missing: d
└── Failure: c, Error: Missing dependency: d
Cached: a = 10
┌── Attempt: c via _ToolSkill(c)
┌── Attempt: b via AutoToolCraft(b)
└── Failure: b, Error: GadgetFailure(None)
┌── Attempt: b via _ToolSkill(b)
│   ┌── Attempt: a via _ToolSkill(a)
│   └── Success: a = 10
└── Success: b = 20
├── Cached: d = 5
└── Success: c = 15
┌── Attempt: a via _ToolSkill(a)
└── Success: a = 10
┌── Attempt: b via AutoToolCraft(b)
└── Failure: b, Error: GadgetFailure(None)
┌── Attem

In [4]:

root = process_log(log)
root

[Node(c), Node(a), Node(b), Node(c), Node(a)]

In [10]:
root[2].followup

[Node(a)]

In [155]:
def print_node_tree(node, prefix='', is_last=True, is_followup=False):
    # Determine the connector
    # is_last = is_last or node.followup is not None
    connector = '└── ' if is_last else '├── '
    connector = '│   ' if is_followup else connector

    # Print the attempt
    print(prefix + connector + f"{node.gizmo} ({node.outcome})")

    # Prepare the new prefix for child nodes
    if is_last:
        child_prefix = prefix + '    '
    else:
        child_prefix = prefix + '│   '

    # Print each child
    count = len(node.children)
    for i, child in enumerate(node.children):
        is_last = (i == count - 1)
        print_node_tree(child, child_prefix, is_last)
    
    if node.followup:
        print_node_tree(node.followup, prefix, is_last, True)
        
def print_event_tree(node, prefix='', is_last=True, is_followup=False):
    
    if node.outcome == 'cached':
        print(prefix + '─── ' + f"{node.gizmo} = {node.value}")
        return
    
    starter = '├── ' if is_followup else '┌── '
    
    if node.followup:
        print(prefix + starter + f"{node.gizmo} ({node.outcome})")
        print_event_tree(node.followup, prefix, is_last, True)
    else:
        print(prefix + starter + f"{node.gizmo}")
        prefix += '│   '
        for child in node.children:
            print_event_tree(child, prefix, is_last)
        prefix = prefix[:-4]
        ender = '└── '
        print(prefix + ender + f"{node.outcome}")
        

def print_node_tree(node, prefix='', is_first=True, is_last=True, is_followup=False, *, width=4, printer=None):
    if printer is None:
        printer = lambda node: node.gizmo
    assert width >= 2, 'width must be at least 2'
    # Determine the connector
    # is_last = is_last or node.followup is not None
    if is_first:
        connector = ''
    elif is_last:
        connector = '└' + '─' * (width - 2) + ' '
    elif node.followup is not None:
        connector = '│' + ' ' * (width - 2) + ' '
    else:
        connector = '├' + '─' * (width - 2) + ' '
    # Print the attempt
    yield prefix + connector + printer(node)

    # Prepare the new prefix for child nodes
    if is_first:
        child_prefix = prefix
    elif is_last:
        child_prefix = prefix + ' ' * width
    else:
        child_prefix = prefix + '│' + ' ' * (width - 1)

    # Print each child
    count = len(node.children)
    for i, child in enumerate(node.children):
        is_last = (i == count - 1)
        yield from print_node_tree(child, child_prefix, False, is_last, width=width, printer=printer)
    
    if node.followup:
        yield from print_node_tree(node.followup, prefix, False, is_last, True, width=width, printer=printer)
        

In [156]:
print('\n'.join(print_node_tree(root[0], width=4)))

c
│   b
├── b
│   └── a
└── d


```
┌── c
│   ├── b (failure)
│   │   b
│   │   └── a
│   └── d = 5
└── success

┌── c
│   ┌── b (failure)
│   ├── b
│   │   ┌── a
│   │   └── success
│   └── success
│   ─── d = 5
└── success

c (success)
├── b (failure)
│   b (success)
│   └── a (success)
└── d (cached)

c (success)
├─ b (failure)
│  b (success)
│  └─ a (success)
└─ d (cached)

c (success)
├ b (failure)
│ b (success)
│ └ a (success)
└ d (cached)

In [129]:
def report_time(t):
    units = [
        ('h', 3600, None),
        ('min', 60, 3600),
        ('s', 1, 60),
        ('ms', 1e-3, 1),
        ('µs', 1e-6, 1e-4)
    ][::-1]
    if t == 0:
        return '0 µs'
    for i, (unit_name, unit_scale, next_unit_threshold) in enumerate(units):
        value = t / unit_scale
        formatted_value = format_sig_figs(value, 2)
        rounded_value = float(formatted_value)

        # Check if the rounded value reaches or exceeds the threshold for the next unit
        if next_unit_threshold is not None:
            next_unit_value = next_unit_threshold / unit_scale
            if rounded_value > next_unit_value:
                continue  # Move to the next larger unit

        return f"{formatted_value} {unit_name}"
    # If none of the units matched, default to the largest unit
    value = t / units[-1][1]
    formatted_value = format_sig_figs(value, 2)
    return f"{formatted_value} {units[-1][0]}"

def format_sig_figs(num, sig_figs):
    if num == 0:
        return "0"
    else:
        import math
        order = int(math.floor(math.log10(abs(num))))
        factor = 10 ** (sig_figs - 1 - order)
        rounded_num = round(num * factor) / factor
        decimals = max(sig_figs - order - 1, 0)
        # Avoid unnecessary decimal places if rounded_num is an integer
        if rounded_num == int(rounded_num):
            decimals = 0
        format_string = "{0:." + str(decimals) + "f}"
        return format_string.format(rounded_num)

In [130]:
# Test microseconds
assert report_time(0) == "0 µs"
assert report_time(1e-7) == "0.10 µs"
assert report_time(5e-5) == "50 µs"
assert report_time(9.99e-5) == "100 µs"

# Test milliseconds
assert report_time(1e-4) == "0.10 ms"
assert report_time(0.00015) == "0.15 ms"
assert report_time(0.005) == "5.0 ms"
assert report_time(0.0999) == "100 ms"
assert report_time(0.14122) == "140 ms"
assert report_time(0.999) == "1 s"

# Test seconds
assert report_time(1) == "1.0 s"
assert report_time(1.5) == "1.5 s"
assert report_time(30.5) == "31 s"
assert report_time(59.99) == "60 s"

# Test minutes
assert report_time(60) == "1.0 min"
assert report_time(90) == "1.5 min"
assert report_time(1800) == "30 min"
assert report_time(3599) == "60 min"

# Test hours
assert report_time(3600) == "1.0 h"
assert report_time(7200) == "2.0 h"
assert report_time(10800) == "3.0 h"
assert report_time(86400) == "24 h"

# Test large values
assert report_time(1e6) == "280 h"          # 1 million seconds
assert report_time(2.592e6) == "720 h"      # 30 days in seconds
assert report_time(1e9) == "280,000 h"      # Should not be in scientific notation

# Test small values
assert report_time(1e-10) == "0.00010 µs"
assert report_time(5e-9) == "0.0050 µs"

# Test negative values
assert report_time(-1e-7) == "-0.10 µs"
assert report_time(-0.005) == "-5.0 ms"
assert report_time(-1) == "-1.0 s"
assert report_time(-3600) == "-1.0 h"

# Test edge cases
assert report_time(1e-4) == "0.10 ms"  # Threshold between µs and ms
assert report_time(1) == "1.0 s"       # Threshold between ms and s
assert report_time(60) == "1.0 min"    # Threshold between s and min
assert report_time(3600) == "1.0 h"    # Threshold between min and h

# Test zero
assert report_time(0) == "0 µs"

# Test precision
assert report_time(0.123456) == "120 ms"
assert report_time(12.3456) == "12 s"
assert report_time(123.456) == "120 s"
assert report_time(1234.56) == "21 min"

# Test no scientific notation
result = report_time(999)
assert "e" not in result, "Output contains scientific notation"
result = report_time(1e6)
assert "e" not in result, "Output contains scientific notation"


AssertionError: 

In [135]:
report_time(104e-6)

'100 µs'